<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 7</font>
## <font color='blue'>Construindo Base de Conhecimento Para o LLM com Vector Database</font>

Obs: Se tiver dificuldade para executar o projeto localmente, execute no Google Colab.

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
%env TOKENIZERS_PARALLELISM=True

env: TOKENIZERS_PARALLELISM=True


In [3]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [4]:
!pip install -q -r requirements.txt

Nota: Para reproduzir o projeto use as versões dos pacotes conforme indicado ao final do capítulo. O projeto não foi testado em outras versões. Você é livre para usar a versão que desejar, mas nesse caso também é responsável por pesquisar a documentação e ajustar o código conforme necessário. Nosso objetivo aqui não é estudar versão de pacote e sim o processo de construção da solução baseada em LLM.

In [5]:
# Imports
import pypdf
import chromadb
import urllib3
import accelerate
import sentence_transformers
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

## Extraindo Dados de Texto de Arquivos PDF

In [6]:
# Cria um objeto para carregar o arquivo PDF
dsa_loader = PyPDFLoader("arquivos/ArtigoDSA1.pdf")

In [7]:
type(dsa_loader)

langchain_community.document_loaders.pdf.PyPDFLoader

In [8]:
# Carrega o arquivo PDF
pages = dsa_loader.load()

In [9]:
pages

[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-18T06:02:27+00:00', 'author': 'DM PM', 'moddate': '2023-12-18T06:02:27+00:00', 'source': 'arquivos/ArtigoDSA1.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='A Habilidade Mais Importante na Era da Inteligência Artificial \n \nA pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que \ntudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super \npositivo.  \nPara dezenas de milhões de trabalhadores, não. \nEles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os \ncontadores, os digitadores, os secretários executivos, procurando trabalho em uma nova \neconomia na qual as pessoas contratadas têm títulos como “Engenheiro de Nuv em” ou “Hacker \nde Crescimento” em seus currículos. Sem um esforço concentrado para retreiná-los, descobriram \nos pesquisadores da RAND Europe, eles 

In [10]:
# Vamos visualizar o conteúdeo de apenas uma página, neste caso da página 3 (indexação em Python começa por 0)
page = pages[2]

In [11]:
print("Conteúdo da Página:", page.page_content[0:700])

Conteúdo da Página: Não importa sua área, seu mercado, sua graduação, sua idade ou seu gênero. O mundo está 
passando por uma profunda transformação digital e os empregos como conhecemos estão sendo 
reinventados. Aqueles que não acompanharem essa evolução natural ficarão para trás, como 
tantas vezes vimos na história humana. Aprenda o máximo que puder, sobre diferentes temas, 
desde habilidades interpessoais até habilidades técnicas. O único limite sobre o que você pode 
aprender é o que você impõe a si mesmo. 
“Seja Bom em Aprender”. Mantenha-se em modo constante de aprendizado. 
Equipe DSA 
www.datascienceacademy.com.br


In [12]:
print("Metadados da Página:", page.metadata)

Metadados da Página: {'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-18T06:02:27+00:00', 'author': 'DM PM', 'moddate': '2023-12-18T06:02:27+00:00', 'source': 'arquivos/ArtigoDSA1.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}


## Divisão dos Dados de Texto em Chunks

In [13]:
# Cria o separador de chunks de texto
dsa_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)

**chunk_size = 1000**: Define que cada pedaço de texto resultante terá no máximo 1000 caracteres.

**chunk_overlap = 20**: Indica que cada chunk terá 20 caracteres de sobreposição com o chunk seguinte. Isso significa que os últimos 20 caracteres de um chunk serão repetidos no início do próximo chunk.

Para que serve:

Essa abordagem é útil em várias situações onde textos grandes precisam ser processados ou analisados, como:

- Entrada para modelos de linguagem: Muitos LLMs têm um limite de tokens que podem processar em uma única interação. Dividir o texto em pedaços menores garante que o texto seja enviado dentro do limite permitido.

- Análise e indexação de dados: É comum em mecanismos de busca e pipelines de processamento de dados, onde dividir o texto em pedaços menores facilita a indexação e recuperação de informações.

- Manutenção de contexto: Quando o processamento envolve documentos longos, essa técnica permite lidar com eles de forma mais eficiente, ao dividir as partes sem perder lógica ou coesão.

In [14]:
# Aplica o objeto e extrai os chunks (documentos)
docs = dsa_splitter.split_documents(pages)

In [15]:
print("Total de Chunks (Documentos):", len(docs))

Total de Chunks (Documentos): 7


In [16]:
print("Conteúdo do Último Chunk (Documento):", docs[6])

Conteúdo do Último Chunk (Documento): page_content='Não importa sua área, seu mercado, sua graduação, sua idade ou seu gênero. O mundo está 
passando por uma profunda transformação digital e os empregos como conhecemos estão sendo 
reinventados. Aqueles que não acompanharem essa evolução natural ficarão para trás, como 
tantas vezes vimos na história humana. Aprenda o máximo que puder, sobre diferentes temas, 
desde habilidades interpessoais até habilidades técnicas. O único limite sobre o que você pode 
aprender é o que você impõe a si mesmo. 
“Seja Bom em Aprender”. Mantenha-se em modo constante de aprendizado. 
Equipe DSA 
www.datascienceacademy.com.br' metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-18T06:02:27+00:00', 'author': 'DM PM', 'moddate': '2023-12-18T06:02:27+00:00', 'source': 'arquivos/ArtigoDSA1.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}


## Carregando os Vetores dos Dados de Texto no Banco de Dados Vetorial

O código implementa um sistema de busca semântica utilizando um banco de dados vetorial (vectordb) para identificar os pontos mais relevantes em relação a uma pergunta, baseado na similaridade semântica entre os documentos e a questão fornecida. 

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://www.trychroma.com/

In [17]:
# Cria o banco de dados vetorial
vectordb = Chroma.from_documents(documents = docs,
                                 embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2"),
                                 persist_directory = "dsavectordb/chroma/")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Chroma.from_documents(documents=docs)**: Cria um banco de dados vetorial utilizando os documentos fornecidos (armazenados na variável docs). Esses documentos podem ser textos, artigos ou qualquer tipo de dado textual que você deseja indexar.

**HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")**: Usa o modelo de embeddings semânticos chamado all-MiniLM-L6-v2, da Hugging Face, para transformar os textos em vetores numéricos. Embeddings são representações matemáticas dos textos que capturam seu significado semântico.

**persist_directory="dsavectordb/chroma/"**: Especifica o diretório onde o banco de dados vetorial será salvo (persistido), para que possa ser reutilizado em sessões futuras sem a necessidade de reprocessar os documentos.

In [18]:
# Total de coleções no vector db
vectordb._collection.count()

7

## Testando os Parâmetros da Busca Vetorial

In [19]:
# Define uma questão
questao = "A pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo?"

In [20]:
# Realiza a busca vetorial
pontos_relevantes = vectordb.max_marginal_relevance_search(questao, k = 2, fetch_k = 3)

**max_marginal_relevance_search()**: Realiza uma busca no banco de dados vetorial com base na relevância marginal máxima (MMR - Maximal Marginal Relevance). Essa técnica é usada para encontrar documentos relevantes para a questão fornecida, diminuindo a redundância nas respostas. Em vez de retornar documentos muito semelhantes entre si, ela garante diversidade nas respostas enquanto mantém a relevância. Leia o manual em pdf no Capítulo 16 com mais detalhes.

Parâmetros:

**questao**: A questão em texto natural usada para calcular a similaridade semântica com os documentos do banco de dados vetorial.

**k=2**: Define o número de documentos finais que serão retornados como os mais relevantes.

**fetch_k=3**: Especifica que o algoritmo deve buscar inicialmente os 3 documentos mais relevantes e, em seguida, aplicar a técnica de MMR para selecionar os 2 mais diversificados e relevantes.

In [21]:
print(pontos_relevantes[0])

page_content='A Habilidade Mais Importante na Era da Inteligência Artificial 
 
A pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que 
tudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super 
positivo.  
Para dezenas de milhões de trabalhadores, não. 
Eles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os 
contadores, os digitadores, os secretários executivos, procurando trabalho em uma nova 
economia na qual as pessoas contratadas têm títulos como “Engenheiro de Nuv em” ou “Hacker 
de Crescimento” em seus currículos. Sem um esforço concentrado para retreiná-los, descobriram 
os pesquisadores da RAND Europe, eles provavelmente serão deixados para trás. 
E não apenas eles. O custo dessa crescente lacuna de habilidades será medido em trilhões de 
dólares e recairá mais fortemente em lugares que não possuem infraestrutura digital confiável,' metadata={'total_pages': 3, 'creator': '

## Definindo o LLM

https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct

In [22]:
# Define o nome do LLM conforme consta no HF
nome_modelo_llm = "Qwen/Qwen2.5-1.5B-Instruct"

In [23]:
# Carrega o modelo
modelo = AutoModelForCausalLM.from_pretrained(nome_modelo_llm,
                                              torch_dtype = "auto",
                                              device_map = "auto")

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [24]:
# Carrega o tokenizador do modelo
tokenizer = AutoTokenizer.from_pretrained(nome_modelo_llm)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

## Configurando o Contexto

In [25]:
# Define a questão
questao = "A pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo?"

In [26]:
# Extrai o contexto da questão (ou seja, executa a busca vetorial)
contexto = vectordb.max_marginal_relevance_search(questao, k = 2, fetch_k = 3)

## Configurando o Prompt

In [27]:
# Cria o prompt
prompt = f"""
Você é um assistente especialista. Você usa o contexto fornecido como sua base de conhecimento complementar para responder à pergunta.
context = {contexto}
question = {questao}
answer =
"""

In [28]:
# Cria a lista de mensagens de systema e de usuário
messages = [
    {"role": "system", "content": "Você é Qwen, criado pela Alibaba Cloud. Você é um assistente especialista."},
    {"role": "user", "content": prompt}
]

## Tokenização do Prompt

In [29]:
# Aplica o chat template
texto = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = True)

In [30]:
texto

"<|im_start|>system\nVocê é Qwen, criado pela Alibaba Cloud. Você é um assistente especialista.<|im_end|>\n<|im_start|>user\n\nVocê é um assistente especialista. Você usa o contexto fornecido como sua base de conhecimento complementar para responder à pergunta.\ncontext = [Document(metadata={'creationdate': '2023-12-18T06:02:27+00:00', 'author': 'DM PM', 'moddate': '2023-12-18T06:02:27+00:00', 'creator': 'Microsoft Word', 'producer': 'PyPDF', 'source': 'arquivos/ArtigoDSA1.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='A Habilidade Mais Importante na Era da Inteligência Artificial \\n \\nA pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que \\ntudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super \\npositivo.  \\nPara dezenas de milhões de trabalhadores, não. \\nEles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os \\ncontadores, os digitadores, os secre

In [31]:
# Aplica a tokenização
model_inputs = tokenizer([texto], return_tensors = "pt").to(modelo.device)

In [32]:
model_inputs

{'input_ids': tensor([[151644,   8948,    198,  69286,   3958,   1207,  16948,     11,  27558,
           2123,  32723,  54364,  14817,     13,  80797,   3958,   4443,   7789,
           6817,  32297,   9087,     13, 151645,    198, 151644,    872,    271,
          69286,   3958,   4443,   7789,   6817,  32297,   9087,     13,  80797,
          33715,    297,  76743,  56009,    757,   5249,   7953,  19345,   2331,
            409,  70483,  15027,  22766,    277,   3348,  64034,   3784,    817,
          59910,    624,   2147,    284,    508,   7524,  54436,  12854,  37375,
           1028,   1210,    364,     17,     15,     17,     18,     12,     16,
             17,     12,     16,     23,     51,     15,     21,     25,     15,
             17,     25,     17,     22,     10,     15,     15,     25,     15,
             15,    516,    364,   3094,   1210,    364,   8395,   5851,    516,
            364,   2593,   1028,   1210,    364,     17,     15,     17,     18,
             1

## Gerando Resposta com o LLM

In [33]:
# Gera resposta com o LLM
generated_ids = modelo.generate(**model_inputs, max_new_tokens = 512)

In [34]:
generated_ids

tensor([[151644,   8948,    198,  69286,   3958,   1207,  16948,     11,  27558,
           2123,  32723,  54364,  14817,     13,  80797,   3958,   4443,   7789,
           6817,  32297,   9087,     13, 151645,    198, 151644,    872,    271,
          69286,   3958,   4443,   7789,   6817,  32297,   9087,     13,  80797,
          33715,    297,  76743,  56009,    757,   5249,   7953,  19345,   2331,
            409,  70483,  15027,  22766,    277,   3348,  64034,   3784,    817,
          59910,    624,   2147,    284,    508,   7524,  54436,  12854,  37375,
           1028,   1210,    364,     17,     15,     17,     18,     12,     16,
             17,     12,     16,     23,     51,     15,     21,     25,     15,
             17,     25,     17,     22,     10,     15,     15,     25,     15,
             15,    516,    364,   3094,   1210,    364,   8395,   5851,    516,
            364,   2593,   1028,   1210,    364,     17,     15,     17,     18,
             12,     16,    

In [35]:
# Descompacta as respostas
# Objetivo: Extrair apenas os tokens gerados pelo modelo (ou seja, a parte da saída que vem após 
# os tokens de entrada). Isso é útil, pois os modelos como GPT ou outros baseados em decodificação 
# autoregressiva frequentemente retornam uma concatenação da entrada e da saída.
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, 
                                                                              generated_ids)]

In [36]:
generated_ids

[tensor([ 14027,     11,    264,  12217,  21925,    653,  19966,     12,     16,
             24,   1613,   7865,    283,   4595,  19488,   8980,    297,  21198,
           6355,    653,  60244,  78669,   7377,    976,  11804,    297,  28352,
             13,  38676,  81636,    902,  33910,   6616,  40834,   1609,   3473,
          15723,   2782,  21679,  24524,   2123,  27525,  13098,  10411,   3590,
           1709,   1102,    283,   2994,   1560,   1064,  19812,  57942,     11,
          22638,   4883,  53760,    436,    264,   9772,    277,  52097,  68332,
          13596, 143565,  47831,   3348,  62786,    627,    300,    389,   8447,
             13, 133036,    827,    704,     11,    264,   4441,   9075,  60839,
           6817,    409,    993,    354,    277,   2048,     84,  15249,  15723,
           2782,   3348,    883,    465,    438,   1997,  33465,  27863,   2782,
          29231,    297,  86624,    409,    922,   1390,   7157,  34601,   5919,
           1963,     84,   3

In [37]:
# Aplica o decode para obter o texto gerado
response = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)[0]

In [38]:
print(response)

Sim, a pandemia do COVID-19 acelerou significativamente o ritmo do desenvolvimento digital em todo o mundo. Este aumento no uso das tecnologias digitais foi causado pelo isolamento social que resultou da crise sanitária, levando muitos a migrar suas atividades diárias para plataformas on-line. Além disso, a necessidade urgente de adotar soluções digitais para manter as operações funcionais durante o período de quarentena também contribuiu para este rápido avanço.


## Sistema de Perguntas e Respostas Usando a Base de Conhecimento

In [39]:
# Define a questão
questao = "Quantos empregos o Fórum Econômico Mundial estima que serão perdidos para automação nos próximos anos?"

# Extrai o contexto da questão (ou seja, executa a busca vetorial)
contexto = vectordb.max_marginal_relevance_search(questao, k = 2, fetch_k = 3)

# Cria o prompt
prompt = f"""
Você é um assistente especialista. Você usa o contexto fornecido como sua base de conhecimento compplementar para responder à pergunta.
context = {contexto}
question = {questao}
answer =
"""

# Cria a lista de mensagens de systema e de usuário
messages = [
    {"role": "system", "content": "Você é Qwen, criado pela Alibaba Cloud. Você é um assistente especialista."},
    {"role": "user", "content": prompt}
]

# Aplica o chat template
texto = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = True)

# Aplica a tokenização
model_inputs = tokenizer([texto], return_tensors = "pt").to(modelo.device)

# Gera resposta com o LLM
generated_ids = modelo.generate(**model_inputs, max_new_tokens = 512)

# Descompacta as respostas
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, 
                                                                              generated_ids)]

# Aplica o decode para obter o texto gerado
response = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)[0]

print(response)                                                                              

De acordo com o documento, o Fórum Econômico Mundial estima que 85 milhões de empregos poderão ser perdidos para automação nos próximos três anos em mais de um duzentos e dezesseis setores.


In [40]:
# Define a questão
questao = "Qual é a habilidade mais importante na era da Inteligência Artificial?"

# Extrai o contexto da questão (ou seja, executa a busca vetorial)
contexto = vectordb.max_marginal_relevance_search(questao, k = 2, fetch_k = 3)

# Cria o prompt
prompt = f"""
Você é um assistente especialista. Você usa o contexto fornecido como sua base de conhecimento compplementar para responder à pergunta.
context = {contexto}
question = {questao}
answer =
"""

# Cria a lista de mensagens de systema e de usuário
messages = [
    {"role": "system", "content": "Você é Qwen, criado pela Alibaba Cloud. Você é um assistente especialista."},
    {"role": "user", "content": prompt}
]

# Aplica o chat template
texto = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = True)

# Aplica a tokenização
model_inputs = tokenizer([texto], return_tensors = "pt").to(modelo.device)

# Gera resposta com o LLM
generated_ids = modelo.generate(**model_inputs, max_new_tokens = 512)

# Descompacta as respostas
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, 
                                                                              generated_ids)]

# Aplica o decode para obter o texto gerado
response = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)[0]

print(response)                                                                              

A habilidade mais importante na era da Inteligência Artificial é aprender constantemente, seja sobre habilidades interpessoais ou técnicas. A capacidade de se adaptar rapidamente às mudanças tecnológicas e manter-se atualizado é crucial para se manter competitivo no mercado de trabalho durante a era da IA.


In [41]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [42]:
#%watermark -v -m

In [43]:
#%watermark --iversions

# Fim